In [39]:
# We will take our Bank Ledger CSV file and our GL CSV file and create dataframes using Pandas
# We will create a 'Concat' column for each dataframe comprised of Date and Sum of Debit and Credit 
# We will merge the dataframes
# We will write a function to identify matches 

In [40]:
import pandas as pd 
import numpy as np

In [41]:
import os

In [42]:
os.getcwd()

'C:\\Users\\12014\\OneDrive\\Documents\\ledger project'

In [43]:
os.chdir('C:\\Users\\12014\\OneDrive\\Documents\\ledger project')

In [44]:
# Create bank file dataframe 

bank=pd.read_csv("bankStatement.csv")
bank.head()

,Statement Value Date,Debit,Credit,Statement Details Info
0,1/1/2021,10,0,TEST ENTRY 01
1,1/5/2021,0,-15,TEST ENTRY 02
2,1/15/2021,1,0,TEST ENTRY 03
3,1/23/2021,67,0,TEST ENTRY 04
4,1/23/2021,54,0,TEST ENTRY 05


In [45]:
# Create concatenated column based on Date and Sum of Debit and Credit

bank['sum'] = bank['Debit'].round(2)+bank['Credit'].round(2)
bank['concat'] = bank['Statement Value Date'] + ' ' + bank['sum'].astype(str)
bank.head()

,Statement Value Date,Debit,Credit,Statement Details Info,sum,concat
0,1/1/2021,10,0,TEST ENTRY 01,10,1/1/2021 10
1,1/5/2021,0,-15,TEST ENTRY 02,-15,1/5/2021 -15
2,1/15/2021,1,0,TEST ENTRY 03,1,1/15/2021 1
3,1/23/2021,67,0,TEST ENTRY 04,67,1/23/2021 67
4,1/23/2021,54,0,TEST ENTRY 05,54,1/23/2021 54


In [46]:
# Create General Ledger dataframe

gl=pd.read_csv('ledgerReport.csv')
gl.head()

,Transaction Date,Description,Debit Amount,Credit Amount
0,1/1/2021,abcd,11,0
1,1/15/2021,qwerty,1,0
2,1/23/2021,zxcvb,65,0
3,1/23/2021,asdfg,54,0
4,1/25/2021,1234,0,-5


In [47]:
# Checking column names 

print(gl.columns.tolist())

['Transaction Date', 'Description ', 'Debit Amount', 'Credit Amount']


In [48]:
gl['sum'] = gl['Debit Amount'].round(2)+gl['Credit Amount'].round(2)
gl['concat']=gl['Transaction Date'] + ' ' + gl['sum'].astype(str)
gl.head()

,Transaction Date,Description,Debit Amount,Credit Amount,sum,concat
0,1/1/2021,abcd,11,0,11,1/1/2021 11
1,1/15/2021,qwerty,1,0,1,1/15/2021 1
2,1/23/2021,zxcvb,65,0,65,1/23/2021 65
3,1/23/2021,asdfg,54,0,54,1/23/2021 54
4,1/25/2021,1234,0,-5,-5,1/25/2021 -5


In [49]:
# Dropping columns to see only important info

bank = bank[['sum', 'concat']]
bank.head()

,sum,concat
0,10,1/1/2021 10
1,-15,1/5/2021 -15
2,1,1/15/2021 1
3,67,1/23/2021 67
4,54,1/23/2021 54


In [50]:
gl = gl[['sum', 'concat']]
gl.head()

,sum,concat
0,11,1/1/2021 11
1,1,1/15/2021 1
2,65,1/23/2021 65
3,54,1/23/2021 54
4,-5,1/25/2021 -5


In [51]:
result = pd.merge(bank,gl,how='outer',on=['concat'],indicator='status')
result.head(15)

,sum_x,concat,sum_y,status
0,10.0,1/1/2021 10,NaN,left_only
1,-15.0,1/5/2021 -15,NaN,left_only
2,1.0,1/15/2021 1,1.0,both
3,67.0,1/23/2021 67,NaN,left_only
4,54.0,1/23/2021 54,54.0,both
5,-5.0,1/25/2021 -5,-5.0,both
6,-40.0,1/31/2021 -40,NaN,left_only
7,NaN,1/1/2021 11,11.0,right_only
8,NaN,1/23/2021 65,65.0,right_only
9,NaN,1/30/2021 -40,-40.0,right_only


In [52]:
form = {'left_only':"Found in Bank, does not match with GL",
       'right_only': "Found in GL, does not match with Bank",
       "both": 'Found in both, matching!'}
result['status'] = result['status'].map(form).astype(str)

In [53]:
result.fillna(0, inplace=True)
print(result)

   sum_x         concat  sum_y                                 status
0   10.0    1/1/2021 10    0.0  Found in Bank, does not match with GL
1  -15.0   1/5/2021 -15    0.0  Found in Bank, does not match with GL
2    1.0    1/15/2021 1    1.0               Found in both, matching!
3   67.0   1/23/2021 67    0.0  Found in Bank, does not match with GL
4   54.0   1/23/2021 54   54.0               Found in both, matching!
5   -5.0   1/25/2021 -5   -5.0               Found in both, matching!
6  -40.0  1/31/2021 -40    0.0  Found in Bank, does not match with GL
7    0.0    1/1/2021 11   11.0  Found in GL, does not match with Bank
8    0.0   1/23/2021 65   65.0  Found in GL, does not match with Bank
9    0.0  1/30/2021 -40  -40.0  Found in GL, does not match with Bank


In [56]:
result.to_csv('Reconcilitation.csv')